In [ ]:
import concurrent.futures
import os
import sys
from functools import partial

sys.path.append('..')

import numpy as np
import scipy as sp

from safe_module import distance
from themes import correlation_plot
from themes import global_plot

In [ ]:
write = True

In [ ]:
# Analysis for -
# 1. - MSAT
# 2. - Becker

study = "Becker"
sigma = "ORTH"
compute = False

In [ ]:
if study=="MSAT":
    
    file_path = "../data/TX2008.V1.P010.dat"
    data = np.loadtxt(file_path)
    n_material = np.size(data,0)
    lons = data[:n_material,1]
    lats = data[:n_material,0]
    cijs = data[:n_material,3:24]
    
    if sigma == 'XISO':
        i = 0
    
elif study=="Becker":
    
    file_path = "../data/safs417nc3_er.s.0.75.WLD.200.savd.dat"
    data = np.loadtxt(file_path)
    n_material = np.size(data,0)
    lons = data[:n_material,0]
    lats = data[:n_material,1]
    cijs = data[:n_material,3:24]
    
    if sigma == 'XISO':
        i = 0
        table_index = 1
        cmap = 'RdYlBu'
        vmin = 0
        vmax = 9
    
    elif sigma == 'ORTH':
        i = 1
        table_index = 3
        cmap = 'PiYG'
        vmin = 0
        vmax = 2

In [ ]:
if compute:

    c_vecs = [cijs[i,:] for i in range(n_material)]
    
    symmetry_classes = [sigma]
    n_symmetry_classes = 1
    num_workers = os.cpu_count()
    
    betas = np.empty((n_symmetry_classes, n_material))
    thetas = np.empty((n_symmetry_classes, n_material))
    sigmas = np.empty((n_symmetry_classes, n_material))
    phis = np.empty((n_symmetry_classes, n_material))
    
    tracker = np.arange(n_material)
    
    for i, sigma in enumerate(symmetry_classes):
    
        print(f'running minimizer for symmetry class: {sigma}')
    
        distance_new = partial(distance, use_parallel_processing=False)
        with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
            futures = list(executor.map(distance_new, c_vecs, [sigma]*n_material, tracker))
    
        for j in range(n_material):
            betas[i,j], thetas[i,j], sigmas[i,j], phis[i,j] = futures[j]
            
    if write:
        np.save(f'betas_{study}', betas)
        np.save(f'thetas_{study}', thetas)
        np.save(f'sigmas_{study}', sigmas)
        np.save(f'phis_{study}', phis)
    
else:
    
    betas = np.load(f"Data/{study}/betas_{study}.npy")
    thetas = np.load(f"Data/{study}/thetas_{study}.npy")
    sigmas = np.load(f"Data/{study}/sigmas_{study}.npy")
    phis = np.load(f"Data/{study}/phis_{study}.npy")

In [ ]:
if study=="MSAT":

    if sigma != 'XISO':
        print("WARNING: some of the color scales, figure titles, and filenames are hardcoded and might be incorrect")
        sigma2 = sigma
    else:
        sigma2 = "hexagonality"

    # plot for betas
    parameter = betas[i,:]
    title = fr'$\beta_{{{sigma}}}$ (°)'
    filename = f'betas_{sigma}'
    global_plot(lons, lats, parameter, title, write, filename, vmin=0, vmax=2.0, extend='max')
    x = parameter

    # plot for thetas
    parameter = abs(np.cos(thetas[i,:]))
    title = '|cos (azimuthal angle)|'
    filename = 'azimuthal_angle'
    global_plot(lons, lats, parameter, title, write, filename, reverse_cmap=True, vmin=0, vmax=1, extend='neither')

    # plot for thetas with points corresponding to sub-vertical phis masked    
    parameter = abs(np.cos(thetas[i,:]))
    parameter[np.cos(2*phis[i,:])>0.5] = np.nan
    title = '|cos (azimuthal angle)|'
    filename = 'azimuthal_angle_subverticals_masked'
    global_plot(lons, lats, parameter, title, write, filename, reverse_cmap=True, vmin=0, vmax=1, extend='neither')
    
    # plot for sigmas
    parameter = sigmas[i,:]*180/np.pi
    title = r'$\sigma$ (°)'
    global_plot(lons, lats, parameter, title, reverse_cmap=True)
    
    # plot for phis
    parameter = np.cos(2*phis[i,:])
    title = f'cos (2 * polar angle)'
    filename = 'polar_angle'
    global_plot(lons, lats, parameter, title, write, filename, reverse_cmap=True, vmin=-1, vmax=1, extend='neither', 
                ticks=np.arange(-1,1.1,0.2))
    
    # plot for anisotropy index
    data_external = sp.io.loadmat(r'Data/MSAT/anisotropy_index_MSAT.mat')
    parameter = np.array(data_external['C'])
    title = 'universal_elastic_anisotropy_index'
    filename = title
    global_plot(lons, lats, parameter, title, write, filename, reverse_cmap=True, vmin=0, vmax=0.1, extend='max')
    
    # plot for proportion of hexagonality
    data_external = sp.io.loadmat(f'Data/MSAT/proportion_of_{sigma2}_MSAT.mat')
    parameter = np.array(data_external['PS'])
    title = f'proportion_of_{sigma2}'
    filename = title
    global_plot(lons, lats, parameter, title, write, filename, reverse_cmap=True, vmin=0, vmax=1, extend='neither')
    y = parameter.reshape(2556)
    
    # plot for hexagonality percentage
    data_external = sp.io.loadmat(f'Data/MSAT/{sigma2}_percentage_MSAT.mat')
    parameter = np.array(data_external['SP']) * 100
    title = f'{sigma2}_percentage'
    filename = title
    global_plot(lons, lats, parameter, title, write, filename, reverse_cmap=True, vmin=0, vmax=4, extend='max')
    z = parameter.reshape(2556)
    
    # plot for correlation between betas and proportion of hexagonality
    xlabel = f'proportion of {sigma2}'
    ylabel = fr'$\beta_{{{sigma}}}$ (°)'
    figname  = f'beta_{sigma}_vs_proportion_of_{sigma2}.png'
    correlation_plot(y, x, xlabel, ylabel, write, figname)
    
    # plot for correlation between betas and hexagonality percentage
    xlabel = f'{sigma2} percentage'
    ylabel = fr'$\beta_{{{sigma}}}$ (°)'
    figname  = f'beta_{sigma}_vs_{sigma2}_percentage.png'
    correlation_plot(z, x, xlabel, ylabel, write, figname)
    
    # plot for correlation between proportion of hexagonality and hexagonality percentage
    xlabel = f'{sigma2} percentage'
    ylabel = f'proportion of {sigma2}'
    figname  = f'proportion_of_{sigma2}_vs_{sigma2}_percentage.png'
    correlation_plot(z, y, xlabel, ylabel, write, figname)

In [ ]:
if study=="Becker":
    
    if sigma != 'XISO':
        print("WARNING: some of the color scales, figure titles, and filenames are hardcoded and might be incorrect")
        sigma2 = sigma
    else:
        sigma2 = "hexagonality"
    
    # plot for betas
    parameter = betas[i,:]
    title = fr'$\beta_{{{sigma}}}$ (°)'
    filename = f'betas_{sigma}'
    if sigma == 'XISO':
        global_plot(lons, lats, parameter, title, write, filename, central_longitude=180, reverse_cmap=False, cmap=cmap, vmin=0, vmax=1.2, extend='max')
    elif sigma == 'ORTH':
        global_plot(lons, lats, parameter, title, write, filename, central_longitude=180, reverse_cmap=False, cmap=cmap, vmin=0, vmax=0.4, extend='max')
    x = parameter
    
    # plot for proportion of hexagonality
    data_external = sp.io.loadmat(f'Data/Becker/proportion_of_{sigma2}_Becker.mat')
    parameter = np.array(data_external['PS'])
    title = f'proportion_of_{sigma2}'
    filename = title
    if sigma == 'XISO':
        global_plot(lons, lats, parameter, title, write, filename, central_longitude=180, reverse_cmap=True, cmap=cmap, vmin=0.4, vmax=0.9, extend='both')
    elif sigma == 'ORTH':
        global_plot(lons, lats, parameter, title, write, filename, central_longitude=180, reverse_cmap=True, cmap=cmap, vmin=0.0, vmax=0.4, extend='max')
    y = parameter.reshape(14512)
    
    # plot for hexagonality percentage
    data_external = sp.io.loadmat(f'Data/Becker/{sigma2}_percentage_Becker.mat')
    parameter = np.array(data_external['SP']) * 100
    title = f'{sigma2}_percentage'
    filename = title
    if sigma == 'XISO':
        global_plot(lons, lats, parameter, title, write, filename, central_longitude=180, reverse_cmap=True, cmap=cmap, vmin=vmin, vmax=vmax, extend='neither')
    elif sigma == 'ORTH':
        global_plot(lons, lats, parameter, title, write, filename, central_longitude=180, reverse_cmap=True, cmap=cmap, vmin=vmin, vmax=vmax, extend='max')
    z = parameter.reshape(14512)
    
    # plot for provided hexagonality percentage
    data_external = np.loadtxt(r'Data/Becker/tens.200.dat')
    parameter = np.array(data_external[:n_material,table_index]) * 100
    title = f'{sigma2}_percentage_from_table'
    filename = title
    if sigma == 'XISO':
        global_plot(lons, lats, parameter, title, write, filename, central_longitude=180, reverse_cmap=True, cmap=cmap, vmin=vmin, vmax=vmax, extend='neither')
    elif sigma == 'ORTH':
        global_plot(lons, lats, parameter, title, write, filename, central_longitude=180, reverse_cmap=True, cmap=cmap, vmin=vmin, vmax=vmax, extend='max')
    zz = parameter
    
    # plot for correlation between betas and proportion of hexagonality
    xlabel = f'proportion of {sigma2}'
    ylabel = fr'$\beta_{{{sigma}}}$ (°)'
    figname  = f'beta_{sigma}_vs_proportion_of_{sigma2}.png'
    correlation_plot(y, x, xlabel, ylabel, write, figname)
    
    # plot for correlation between betas and hexagonality percentage
    xlabel = f'{sigma2} percentage'
    ylabel = fr'$\beta_{{{sigma}}}$ (°)'
    figname  = f'beta_{sigma}_vs_{sigma2}_percentage.png'
    correlation_plot(z, x, xlabel, ylabel, write, figname)
    
    # plot for correlation between proportion of hexagonality and hexagonality percentage
    xlabel = f'{sigma2} percentage'
    ylabel = f'proportion of {sigma2}'
    figname  = f'proportion_of_{sigma2}_vs_{sigma2}_percentage.png'
    correlation_plot(z, y, xlabel, ylabel, write, figname)
    
    # plot for correlation between provided hexagonality percentage and hexagonality percentage
    xlabel = f'{sigma2} percentage'
    ylabel = f'provided {sigma2} percentage'
    figname  = f'provided_{sigma2}_percentage_vs_{sigma2}_percentage.png'
    correlation_plot(z, zz, xlabel, ylabel, write, figname)